In [32]:
# Dependencies and Setup
# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_p1 import g_key

from pprint import pprint

In [33]:
# Read the .csv file with wildfire location
fire_df = pd.read_csv("Resources/FW_Veg_Rem_Combined.csv")

# Data Cleaning
# Gathering records from California, greater than or equal to 10 acres, and with valid weather data
fire_df=fire_df.loc[(fire_df["state"]=="CA")&
                    (fire_df["fire_size"]>=10)&
                    (fire_df["weather_file"]!="File Not Found")&
                    (fire_df["Temp_pre_30"]!=0)&
                    (fire_df["Temp_pre_15"]!=0)&
                    (fire_df["Temp_pre_7"]!=0)&
                    (fire_df["Temp_cont"]!=0)]
# Drop duplicates
fire_df=fire_df.drop_duplicates(subset=["fire_name","latitude","fire_size"])

fire_df.head()

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
78,78,78,RAMP,450.0,E,Equipment Use,34.330000,-117.513056,CA,7/22/2006,...,2.237500,44.706207,42.983379,49.000000,39.375000,0.0,0.0,0.0,0.0,0.472150
159,159,159,"""PARRA, B""",200.0,D,Equipment Use,39.965000,-122.318056,CA,8/22/1998,...,1.140741,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.518136
608,608,608,BAR,15.0,C,Lightning,39.495278,-121.125833,CA,5/4/2013,...,2.492233,38.636551,29.997067,28.645233,43.261307,0.0,0.0,0.0,0.0,0.506099
733,733,733,NaN,912.0,E,Missing/Undefined,34.503787,-118.616166,CA,5/19/2013,...,2.738095,52.983668,56.235149,51.769663,63.463980,0.0,0.0,0.0,0.0,0.483234
764,764,764,RIVER 2,10.0,C,Campfire,37.355000,-118.341111,CA,5/31/2004,...,3.718890,26.371394,27.932203,30.531579,27.397875,0.0,0.0,0.0,2.6,0.478753


In [35]:
fire_df.sort_values(by=['fire_size'], ascending=False).head(10)

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
23098,23098,23098,RUSH,315578.8,G,Lightning,40.615300,-120.115300,CA,8/12/2012,...,1.911795,33.866022,30.822222,28.764706,32.930769,0.0,0.0,0.0,1.5,0.495251
52048,52048,52048,CEDAR,280059.0,G,Campfire,32.985556,-116.727500,CA,10/25/2003,...,2.222623,65.414336,57.996875,44.378049,67.385714,0.0,0.0,0.0,1.3,0.147973
53138,53138,53138,WITCH,197990.0,G,Miscellaneous,33.074444,-116.711111,CA,10/21/2007,...,4.501634,67.395706,70.626134,79.041812,34.361842,6.0,4.9,4.1,0.0,0.146632
53372,53372,53372,BASIN COMPLEX,162818.0,G,Lightning,36.281111,-121.658333,CA,6/21/2008,...,2.435183,67.747265,67.467593,70.374408,79.686735,0.0,0.0,0.0,0.0,0.139513
52715,52715,52715,DAY,162702.0,G,Debris Burning,34.632222,-118.769722,CA,9/4/2006,...,2.304438,72.910591,76.310811,76.139535,77.432584,0.0,0.0,0.0,0.0,0.087243
53634,53634,53634,STATION,160371.0,G,Arson,34.242500,-118.188889,CA,8/26/2009,...,2.583182,29.573143,30.027607,33.027027,25.271331,0.0,0.0,0.0,0.0,0.103064
55037,55037,55037,ROUGH,151623.0,G,Lightning,36.873611,-118.904722,CA,7/31/2015,...,2.340721,47.596154,44.543825,40.995851,48.982707,1.8,0.0,0.0,0.0,0.084201
51782,51782,51782,MCNALLY,150696.0,G,Campfire,35.873611,-118.458056,CA,7/21/2002,...,2.405010,42.076773,41.437080,43.223158,44.155534,0.0,0.0,0.0,178.1,0.072267
54846,54846,54846,FRYING PAN,115279.2,G,Lightning,41.772500,-123.355833,CA,8/11/2014,...,2.772549,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.186989
52161,52161,52161,SIMI INCIDENT,108204.0,G,Missing/Undefined,34.384722,-118.667500,CA,10/25/2003,...,1.587437,48.250000,0.000000,0.000000,27.000000,0.0,0.0,0.0,0.0,0.093175


In [36]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [38]:
# find out the center of map:
lat_m = fire_df.latitude.mean()
lng_m = fire_df.longitude.mean()
print(lat_m,lng_m)

37.20752169508403 -119.92705225686274


In [49]:
# Create a fire size Heatmap layer to the map

locations = fire_df[["latitude","longitude"]]

heatmap_layer = gmaps.heatmap_layer(locations, weights = fire_df["fire_size"], 
                                    dissipating = False, max_intensity = float(fire_df.fire_size.max()), 
                                    point_radius = 0.5)
fig = gmaps.figure(center = (37.2,-119.9), zoom_level = 5.3)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))